In [22]:
import os
import json
import pandas as pd

In [23]:
json_dir = './datasets'
merged_data = []

for filename in os.listdir(json_dir):
    if filename.endswith('.json') and filename != '6_dataset_full.json':
        file_path = os.path.join(json_dir, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            if isinstance(data, list):
                merged_data.extend(data)  # Flatten list
            else:
                merged_data.append(data)

output_file = os.path.join(json_dir, '6_dataset_full.json')
with open(output_file, 'w') as file:
    json.dump(merged_data, file, indent=4)

In [24]:
# Convert merged_data list to a pandas DataFrame
df_merged = pd.DataFrame(merged_data)

df_merged.head()

,id,text,to_mask
0,1,Hej! Jag heter Anna Svensson och vill flytta m...,"[{'label': '[NAMN]', 'value': 'Anna Svensson'}..."
1,2,"Jag har nyligen flyttat till Storgatan 12, 123...","[{'label': '[ADRESS]', 'value': 'Storgatan 12,..."
2,3,Jag vill ändra mina kontaktuppgifter. Mitt nya...,"[{'label': '[TELEFONNUMMER]', 'value': '070-12..."
3,4,Jag försökte logga in med mitt passnummer AB12...,"[{'label': '[PASSNUMMER]', 'value': 'AB1234567..."
4,5,Jag behöver uppdatera mitt bankkontonummer. De...,"[{'label': '[BANKKONTONUMMER]', 'value': 'SE98..."


In [25]:
count_num_ids = df_merged["id"].nunique()
print(f"Number of unique ids: {count_num_ids}")

Number of unique ids: 700


In [26]:
# Count unique labels
count_num_labels = df_merged["to_mask"]

distinct_labels = (
    df_merged["to_mask"]
    .explode()
    .apply(lambda x: x["label"] if isinstance(x, dict) else None)
    .dropna()
    .unique()
)
print(distinct_labels)
print(f"Number of unique labels: {len(distinct_labels)}")

['[NAMN]' '[PERSONNUMMER]' '[ORG-NUMMER]' '[ADRESS]' '[ADRESSHISTORIA]'
 '[TELEFONNUMMER]' '[E-POST]' '[PASSNUMMER]' '[IP-ADRESS]'
 '[BANKKONTONUMMER]' '[ARBETSPLATS]' '[ANSTÄLLNINGSTID]' '[CIVILSTAND]'
 '[POSTNUMMER]' '[KREDITKORTNUMMER]' '[BANKGIRONUMMER]' '[ARBETSGIVENHET]'
 '[TITEL]' '[FÖDELSEDAG]' '[NATIONALITET]' '[FÖDELSEPLATS]' '[ETNICITET]'
 '[SOCIALA_MEDIER_PROFILER]' '[ÅLDER]' '[KÖN]' '[CLEARINGNUMMER]' '[IBAN]'
 '[BIC]' '[STAD]' '[MEDARBETARID]' '[RELIGION]' '[LÄN]'
 '[SKATTEREGISTRERINGSNR]' '[LAND_TILLDELAT_SKATTEREGISTRERINGSNR]'
 '[PLUSGIRO]' '[FÖRSÄKRINGSNUMMER]' '[FÖRSÄKRAD]' '[INBETALNINGSREFERENS]'
 '[UTDELNINGSADRESS]' '[AVTALSSOMRÅDE]' '[AVTALSNUMMER]'
 '[PENSIONSÅLDER_KONTAKTPERSON]' '[STYRELSENS_SÄTE]' '[TIN]'
 '[FÖDELSELAND]' '[LEI]' '[GIIN]' '[AKTIEINNEHAV]']
Number of unique labels: 48


In [27]:
# prettify
from pprint import pprint
import textwrap
import random

In [28]:

# some random support requests and it's corresponding to_mask
random.seed(42)
samples = df_merged[["text", "to_mask"]].sample(10)

for index, row in samples.iterrows():
	wrapped_text = '\n'.join(textwrap.wrap(row['text'], width=80))
	print("\nText:")
	print(wrapped_text)
	print("\nTo Mask:")
	pprint(row['to_mask'])
	print("="*80)
	


Text:
Hej, jag heter Peter Lind och vill öppna ett sparkonto med PlusGiro 678-9012.
Mitt skatteregistreringsnummer är 5566778899 från Sverige.

To Mask:
[{'label': '[NAMN]', 'value': 'Peter Lind'},
 {'label': '[PLUSGIRO]', 'value': '678-9012'},
 {'label': '[SKATTEREGISTRERINGSNR]', 'value': '5566778899'},
 {'label': '[LAND_TILLDELAT_SKATTEREGISTRERINGSNR]', 'value': 'Sverige'}]

Text:
Hej, mitt clearingnummer 1234 verkar inte stämma med mitt nya bankkontonummer
1001 001 001. Behöver ni båda uppgifterna för att koppla kontot?

To Mask:
[{'label': '[CLEARINGNUMMER]', 'value': '1234'},
 {'label': '[BANKKONTONUMMER]', 'value': '1001 001 001'}]

Text:
Vi på PensionFokus AB (org-nummer 557188-3456) behöver kundens medarbetar-ID
PF-7890 för att verifiera pensionsöverföringen. Kundens namn är Erik Larsson.

To Mask:
[{'label': '[ARBETSPLATS]', 'value': 'Alpa Invest'},
 {'label': '[ORG-NUMMER]', 'value': 'Alpa Invest'},
 {'label': '[MEDARBETARID]', 'value': 'PF-7890'},
 {'label': '[NAMN]', 'va

In [29]:
# Get an example of each time the distinct labels are used
df_merged = df_merged.iloc[::-1].reset_index(drop=True)

label_examples = {}

for label in distinct_labels:
    example = df_merged[df_merged["to_mask"].apply(lambda masks: any(d.get("label") == label for d in masks if isinstance(d, dict)))].head(1)
    label_examples[label] = example

for label, example in label_examples.items():
    print(f"Label: {label}")
    print("Text:")
    print(example["text"].values[0])
    print("To Mask:")
    pprint(example["to_mask"].values[0])
    print("="*80)

Label: [NAMN]
Text:
Vi på BusinessVentures AB (org-nummer 557311-0123) vill öppna ett nytt företagskonto för vår klient, Mikael Svensson. Kan ni assistera oss med detta?
To Mask:
[{'label': '[ARBETSPLATS]', 'value': 'Hexa Financial'},
 {'label': '[ORG-NUMMER]', 'value': 'Alpa Invest'},
 {'label': '[NAMN]', 'value': 'Zofia Håkansson'}]
Label: [PERSONNUMMER]
Text:
Jag är 20 år gammal och har precis fyllt 18. Jag vill öppna ett eget sparkonto. Mitt namn är Albin Eriksson och mitt personnummer är 030918-3456.
To Mask:
[{'label': '[ÅLDER]', 'value': '20'},
 {'label': '[NAMN]', 'value': 'Sanja Kovačević'},
 {'label': '[PERSONNUMMER]', 'value': '03091873456'}]
Label: [ORG-NUMMER]
Text:
Vi på BusinessVentures AB (org-nummer 557311-0123) vill öppna ett nytt företagskonto för vår klient, Mikael Svensson. Kan ni assistera oss med detta?
To Mask:
[{'label': '[ARBETSPLATS]', 'value': 'Hexa Financial'},
 {'label': '[ORG-NUMMER]', 'value': 'Alpa Invest'},
 {'label': '[NAMN]', 'value': 'Zofia Håkansso

In [30]:
# Filter and display cases with specific labels
pension_age_cases = df_merged[df_merged["to_mask"].apply(lambda masks: any(d.get("label") == "[FÖRSÄKRAD]" for d in masks if isinstance(d, dict)))]

for _, row in pension_age_cases[["text", "to_mask"]].iterrows():
    wrapped_text = '\n'.join(textwrap.wrap(row['text'], width=80))
    print("\nText:")
    print(wrapped_text)
    print("\nTo Mask:")
    pprint(row['to_mask'])
    print("="*80)



Text:
Vi på FinanceGroup AB (org-nummer 557422-9012) behöver ange försäkringsnummer
för vår anställde, Johan Karlsson. Hans försäkringsnummer är F123456789 och han
är försäkrad förutom sig själv.

To Mask:
[{'label': '[ARBETSPLATS]', 'value': 'FinanceGroup AB'},
 {'label': '[ORG-NUMMER]', 'value': '557422-9012'},
 {'label': '[NAMN]', 'value': 'Johan Karlsson'},
 {'label': '[FÖRSÄKRINGSNUMMER]', 'value': 'F123456789'},
 {'label': '[FÖRSÄKRAD]', 'value': 'utom sig själv'}]


In [31]:
def replace_labels(masks):
    for m in masks:
        if not isinstance(m, dict):
            continue
        if m.get("label") in ['[LAND_TILLDELAT_SKATTEREGISTRERINGSNR]', '[FÖDELSELAND]', '[NATIONALITET]']:
            m["label"] = "[LAND]"
        elif m.get("label") in ['[UTDELNINGSADRESS]', '[ADRESSHISTORIA]']:
            m["label"] = "[ADRESS]"
        elif m.get("label") in ['[FÖDELSEPLATS]', '[STYRELSENS_SÄTE]']:
            m["label"] = "[STAD]"
        elif m.get("label") in ['[PENSIONSÅLDER_KONTAKTPERSON]']:
            m["label"] = "[NAMN]"
            
    return masks

df_merged["to_mask"] = df_merged["to_mask"].apply(replace_labels)

In [32]:
def remove_labels(masks, labels_to_remove):
    if not isinstance(masks, list):
        return masks
    return [m for m in masks if not (isinstance(m, dict) and m.get("label") in labels_to_remove)]

# Define the labels to remove
labels_to_remove = ['[SOCIALA_MEDIER_PROFILER]', '[AKTIEINNEHAV]', '[FÖRSÄKRAD]']

# Apply the function to remove specified labels
df_merged["to_mask"] = df_merged["to_mask"].apply(lambda x: remove_labels(x, labels_to_remove))

In [33]:
# Verify the changes by printing unique labels after replacement
all_labels = (
    df_merged["to_mask"]
    .explode()
    .apply(lambda x: x.get("label") if isinstance(x, dict) else None)
    .dropna()
    .unique()
)
print(all_labels)
print(f"Number of unique labels: {len(all_labels)}")

['[ARBETSPLATS]' '[ORG-NUMMER]' '[NAMN]' '[E-POST]' '[ÅLDER]'
 '[PERSONNUMMER]' '[MEDARBETARID]' '[IBAN]' '[BIC]' '[CLEARINGNUMMER]'
 '[BANKKONTONUMMER]' '[PASSNUMMER]' '[FÖDELSEDAG]' '[BANKGIRONUMMER]'
 '[CIVILSTAND]' '[ADRESS]' '[TELEFONNUMMER]' '[STAD]' '[ARBETSGIVENHET]'
 '[ANSTÄLLNINGSTID]' '[KREDITKORTNUMMER]' '[IP-ADRESS]' '[LÄN]'
 '[POSTNUMMER]' '[ETNICITET]' '[KÖN]' '[RELIGION]' '[LAND]' '[TITEL]'
 '[TIN]' '[LEI]' '[GIIN]' '[FÖRSÄKRINGSNUMMER]' '[SKATTEREGISTRERINGSNR]'
 '[AVTALSSOMRÅDE]' '[AVTALSNUMMER]' '[INBETALNINGSREFERENS]' '[PLUSGIRO]']
Number of unique labels: 38
